In [1]:
import pandas as pd
import daily_file
import ftp_noaa

ftp = ftp_noaa.ftp_noaa('pub/data/ghcn/daily/all')

In [2]:
def data_prep_func_country_code(row):
        return row.name[:2]

def data_prep_func_year_count(row):
    return row['LastYear'] - row['FirstYear']

def mm_to_inches(row):
    return row['VALUE'] / 25.4

def get_station_snow(st_id):
    ftp = ftp_noaa.ftp_noaa('pub/data/ghcn/daily/all')
    ftp.retrieve_file('{}.dly'.format(st_id))
    file = daily_file.daily_file('{}.dly'.format(st_id))
    epply_df = file.df
    epply_df = epply_df[epply_df['ELEMENT']=='SNOW']
    epply_df = epply_df[epply_df['VALUE']>0]
    epply_df['Snow (inches)'] = epply_df.apply(mm_to_inches, axis=1)
    epply_df.sort_values(by='Snow (inches)', ascending=False)
    epply_df = epply_df.rename(columns={'ID':'StationID'})
    # read the stations file into memory and convert to a DataFrame
    # then perform various cleansing and data prep operations
    stationsDF = pd.read_fwf('ghcnd-stations.txt', header=None, delimiter=' '
                     , widths=[12,9,10,7,3,31,4,4,6]
                     , names=['StationID', 'Latitude', 'Longitude', 'Elevation'
                            ,'State', 'StationName', 'GSN_Flag', 'HCN_CRN_Flag'
                            ,'WMO_ID']
                     , dtypes={'WMO_ID':object},index_col='StationID'
                     )
    stationsDF['CountryCode']= stationsDF.apply(data_prep_func_country_code, axis=1)

    # merge the stations DataFrame with states and countries master data
    countriesDF = pd.read_fwf('ghcnd-countries.txt',header=None,delimiter=' '
                            ,names=['CountryCode','CountryName'])
    stationsDF = stationsDF.reset_index().merge(countriesDF,on='CountryCode',how='left').set_index('StationID')

    statesDF = pd.read_fwf('ghcnd-states.txt',header=None,delimiter=' '
                            ,names=['State','StateName'])
    stationsDF= stationsDF.reset_index().merge(statesDF,on='State',how='left').set_index('StationID')
    epply_df = pd.merge(epply_df, stationsDF, right_index=True, left_on='StationID')
    epply_df.to_csv('{}_snow.csv'.format(st_id, sep='\t'))

In [3]:
get_station_snow('US1NEWS0004')

In [4]:
ftp = ftp_noaa.ftp_noaa('pub/data/ghcn/daily/all')
ftp.retrieve_file('USW00014942.dly')

In [5]:
file = daily_file.daily_file('USW00014942.dly')
epply_df = file.df
epply_df = epply_df[epply_df['ELEMENT']=='SNOW']
epply_df = epply_df[epply_df['VALUE']>0]

C:\Users\Nate\Anaconda3\lib\site-packages\pandas\io\parsers.py:741: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [6]:
def mm_to_inches(row):
    return row['VALUE'] / 25.4

epply_df['Snow (inches)'] = epply_df.apply(mm_to_inches, axis=1)
epply_df.sort_values(by='Snow (inches)', ascending=False)
epply_df = epply_df.rename(columns={'ID':'StationID'})

In [7]:
def data_prep_func_country_code(row):
        return row.name[:2]

def data_prep_func_year_count(row):
    return row['LastYear'] - row['FirstYear']

# read the stations file into memory and convert to a DataFrame
# then perform various cleansing and data prep operations
stationsDF = pd.read_fwf('ghcnd-stations.txt', header=None, delimiter=' '
                 , widths=[12,9,10,7,3,31,4,4,6]
                 , names=['StationID', 'Latitude', 'Longitude', 'Elevation'
                        ,'State', 'StationName', 'GSN_Flag', 'HCN_CRN_Flag'
                        ,'WMO_ID']
                 , dtypes={'WMO_ID':object},index_col='StationID'
                 )
stationsDF['CountryCode']= stationsDF.apply(data_prep_func_country_code, axis=1)

# merge the stations DataFrame with states and countries master data
countriesDF = pd.read_fwf('ghcnd-countries.txt',header=None,delimiter=' '
                        ,names=['CountryCode','CountryName'])
stationsDF = stationsDF.reset_index().merge(countriesDF,on='CountryCode',how='left').set_index('StationID')

statesDF = pd.read_fwf('ghcnd-states.txt',header=None,delimiter=' '
                        ,names=['State','StateName'])
stationsDF= stationsDF.reset_index().merge(statesDF,on='State',how='left').set_index('StationID')

In [8]:
epply_df = pd.merge(epply_df, stationsDF, right_index=True, left_on='StationID')

In [12]:
def get_season(row):
    if row['MONTH'] < 6:
        return str(row['YEAR'] - 1)[2:4] + '-' + str(row['YEAR'])[2:4]
    else:
        return str(row['YEAR'])[2:4] + '-' + str(row['YEAR'] + 1)[2:4]

In [19]:
epply_df['Season'] = epply_df.apply(get_season, axis=1)
epply_df.sort_values(by=['YEAR', 'MONTH'], inplace=True)

In [21]:
epply_df.to_csv('epply_snow.csv', sep='\t')